<a href="https://colab.research.google.com/github/TrickXer/LSTM---Predictive-Maintenance-model/blob/main/LSTM_Predictive_Maintenance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Imoprt libraries**

In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_auc_score

### **Load the dataset `AMZN.csv`**

In [81]:
data = pd.read_csv('/content/predictive_maintenance.csv')
data.dropna()
data.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


### **Required Data**


In [90]:
# data.iloc[:, [0,4]]
features = data[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']]
features.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
0,298.1,308.6,1551,42.8,0
1,298.2,308.7,1408,46.3,3
2,298.1,308.5,1498,49.4,5
3,298.2,308.6,1433,39.5,7
4,298.2,308.7,1408,40.0,9


In [91]:
labels = data[['Failure Type']]
labels.head()

,Failure Type
0,No Failure
1,No Failure
2,No Failure
3,No Failure
4,No Failure


In [92]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [93]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1, 1, 1, ..., 1, 1, 1])

### **Preprocess the data using StandardScaler**

In [94]:
scaler = StandardScaler()
features = scaler.fit_transform(features)
features

array([[-0.95238944, -0.94735989,  0.06818514,  0.28219976, -1.69598374],
       [-0.90239341, -0.879959  , -0.72947151,  0.63330802, -1.6488517 ],
       [-0.95238944, -1.01476077, -0.22744984,  0.94428963, -1.61743034],
       ...,
       [-0.50242514, -0.94735989,  0.59251888, -0.66077672, -1.35034876],
       [-0.50242514, -0.879959  , -0.72947151,  0.85400464, -1.30321671],
       [-0.50242514, -0.879959  , -0.2162938 ,  0.02137647, -1.22466331]])

In [95]:
adasyn = ADASYN(random_state=42)
enn = EditedNearestNeighbours()

pipeline = Pipeline([
    ('adasyn', adasyn),
    ('enn', enn)
])

features, labels = pipeline.fit_resample(features, labels)
features, labels

(array([[ 0.39750346, -0.4081528 , -1.09762073,  2.24840603,  0.07932323],
        [ 1.14744397,  0.60286048, -0.90239008,  0.68346634,  0.91198933],
        [ 1.14744397,  0.5354596 , -0.90239008,  0.73362467, -1.35034876],
        ...,
        [ 0.3304729 ,  1.12129001,  3.12953048, -2.11976263,  1.94532569],
        [-0.59599724, -0.00875512,  3.98598791, -2.35877113,  1.75123726],
        [ 0.1547501 ,  0.90695531,  3.29197404, -2.16509517,  1.9085131 ]]),
 array([0, 0, 0, ..., 5, 5, 5]))

### **Split the data for training and testing, then Convert to PyTorch tensors**

In [96]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

X_train.shape, X_test.shape

(torch.Size([45364, 1, 5]), torch.Size([11342, 1, 5]))

### **Create Dataloader**

In [97]:
batch_size = 32

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### **Define the LSTM model**

In [98]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


input_size = X_train.shape[2]  # Number of features
hidden_size = 64
num_layers = 2
num_classes = len(np.unique(labels))  # Number of failure types

model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)
model.to(device)
model

LSTMClassifier(
  (lstm): LSTM(5, 64, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=64, out_features=6, bias=True)
)

### **Define Loss Function and Optimizer**

In [99]:
learning_rate = 1e-4

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

### **Train the model**

In [100]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for inputs, targets in train_loader:
        # Move tensors to the configured device (CPU or GPU)
        inputs, targets = inputs.to(model.fc.weight.device), targets.to(model.fc.weight.device)

        # Forward pass
        outputs = model(inputs)

        # Regularization to avoid over-fitting
        l2_lambda = 5e-4
        l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
        loss = criterion(outputs, targets) + l2_lambda * l2_norm

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    # Step the scheduler with validation loss (use epoch_loss if no separate validation loss)
    scheduler.step(epoch_loss)

    current_lr = scheduler.optimizer.param_groups[0]['lr']
    print(f'Current Learning Rate: {current_lr:.6f}')

Epoch [1/100], Loss: 1.5654
Current Learning Rate: 0.000100
Epoch [2/100], Loss: 0.8993
Current Learning Rate: 0.000100
Epoch [3/100], Loss: 0.6746
Current Learning Rate: 0.000100
Epoch [4/100], Loss: 0.5895
Current Learning Rate: 0.000100
Epoch [5/100], Loss: 0.5466
Current Learning Rate: 0.000100
Epoch [6/100], Loss: 0.5222
Current Learning Rate: 0.000100
Epoch [7/100], Loss: 0.5037
Current Learning Rate: 0.000100
Epoch [8/100], Loss: 0.4901
Current Learning Rate: 0.000100
Epoch [9/100], Loss: 0.4794
Current Learning Rate: 0.000100
Epoch [10/100], Loss: 0.4703
Current Learning Rate: 0.000100
Epoch [11/100], Loss: 0.4631
Current Learning Rate: 0.000100
Epoch [12/100], Loss: 0.4582
Current Learning Rate: 0.000100
Epoch [13/100], Loss: 0.4508
Current Learning Rate: 0.000100
Epoch [14/100], Loss: 0.4461
Current Learning Rate: 0.000100
Epoch [15/100], Loss: 0.4438
Current Learning Rate: 0.000100
Epoch [16/100], Loss: 0.4387
Current Learning Rate: 0.000100
Epoch [17/100], Loss: 0.4349
Curr

In [103]:
# Load the saved model's state dictionary
model.load_state_dict(torch.load('lstm_model.pth', map_location=torch.device(device)))

<All keys matched successfully>

### **Evaluate model**

In [127]:
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    all_targets = []
    all_predictions = []

    with torch.no_grad():  # No need to calculate gradients during evaluation
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(model.fc.weight.device), targets.to(model.fc.weight.device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    avg_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total

    print("Confusion Matrix:")
    print(confusion_matrix(all_targets, all_predictions))

    print("\nClassification Report:")
    print(classification_report(all_targets, all_predictions))

    print(f'\nTest Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')


evaluate_model(model, test_loader, criterion)

Confusion Matrix:
[[1951    0    5    3    0    0]
 [  19 1328   17   11  195   76]
 [   0    0 1896    0    0   17]
 [   0    7    0 1975    0    0]
 [   0   44    0    0 1843   14]
 [   0    0   75    0    3 1863]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1959
           1       0.96      0.81      0.88      1646
           2       0.95      0.99      0.97      1913
           3       0.99      1.00      0.99      1982
           4       0.90      0.97      0.94      1901
           5       0.95      0.96      0.95      1941

    accuracy                           0.96     11342
   macro avg       0.96      0.95      0.95     11342
weighted avg       0.96      0.96      0.96     11342


Test Loss: 0.1461, Test Accuracy: 95.72%


### **Save model**

In [102]:
def save_model(model, path='lstm_model.pth'):
    # Save the state dictionary of the model
    torch.save(model.state_dict(), path)
    print(f'Model saved to {path}')

# Save the model
save_model(model)

Model saved to lstm_model.pth


In [105]:
test = y_test.tolist()

# Find indices using list comprehension
for val in np.unique(test):
  indices = [i for i, value in enumerate(test) if value == val]
  print(f"{val}: {indices}")

0: [1, 6, 9, 10, 16, 24, 35, 49, 58, 59, 66, 69, 72, 82, 87, 95, 116, 137, 145, 160, 164, 166, 169, 171, 175, 178, 181, 183, 186, 191, 201, 207, 212, 216, 219, 220, 228, 235, 241, 247, 251, 259, 262, 263, 272, 281, 291, 292, 294, 304, 310, 314, 316, 320, 327, 328, 333, 335, 340, 344, 349, 357, 363, 365, 377, 378, 387, 391, 394, 399, 404, 405, 412, 413, 416, 418, 421, 432, 433, 434, 437, 457, 464, 465, 467, 490, 493, 496, 497, 499, 501, 508, 509, 512, 521, 527, 536, 545, 557, 561, 580, 594, 596, 599, 607, 612, 621, 623, 633, 638, 640, 642, 647, 655, 658, 669, 670, 672, 678, 694, 698, 703, 704, 711, 723, 726, 732, 740, 748, 760, 762, 764, 767, 774, 776, 788, 790, 811, 815, 818, 820, 821, 823, 828, 829, 833, 838, 840, 854, 855, 873, 878, 880, 884, 885, 889, 890, 899, 902, 907, 908, 912, 914, 918, 920, 928, 929, 930, 939, 950, 958, 974, 1004, 1008, 1009, 1013, 1016, 1027, 1029, 1030, 1037, 1044, 1045, 1048, 1053, 1060, 1062, 1075, 1083, 1086, 1088, 1097, 1098, 1112, 1119, 1124, 1127, 1134,

In [126]:
sample_data = {
    'Air temperature [K]': [315, 290, 305, 320, 300],
    'Process temperature [K]': [320, 280, 310, 330, 295],
    'Rotational speed [rpm]': [1400, 2200, 1300, 2500, 1350],
    'Torque [Nm]': [55, 40, 50, 90, 45],
    'Tool wear [min]': [25, 5, 15, 230, 10]
}

failure_types = [
    'Heat Dissipation',
    'Power Failure',
    'No Failure',
    'Overstrain Failure',
    'Tool Wear Failure'
]

sample_inputs = pd.DataFrame(sample_data)
sample_inputs = scaler.transform(sample_inputs)

sample_input = X_test[6633]

model.eval()

# Move the sample input to the device where the model is (CPU or GPU)
# sample_input = torch.tensor(sample_input, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(model.fc.weight.device)
sample_input = sample_input.unsqueeze(0).to(model.fc.weight.device)

# Make predictions
with torch.no_grad():  # Disable gradient calculation for inference
    output = model(sample_input)

# Get the predicted class
_, predicted_class = torch.max(output, 1)

output = label_encoder.inverse_transform(predicted_class.cpu().numpy()).item()
features = scaler.inverse_transform(sample_input.cpu().numpy().reshape(1, -1))[0]

print(f"Features: {features}")
print(f"Predicted Class: '{predicted_class.item()}'")
print(f"Predicted Class: '{output}'")

Features: [ 301.14923  310.7582  1307.5686    61.13722  191.35875]
Predicted Class: '2'
Predicted Class: 'Overstrain Failure'
